In [ ]:
import requests
import time
from datetime import datetime
import dateutil.parser
import os
import yaml

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium import plugins



In [ ]:
with open("config.yml", 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
SERVER = 'UCSD'
cfg = cfg[SERVER]

In [ ]:
base_url = cfg['url']

def get (url, params=None, extend=True):
    global base_url, token
    set_token()
    if extend == True:
        url = base_url + url
    headers = {'Content-Type': 'application/json', 'Authorization': f'Token {token}'}

    response = requests.get(url, params=params, headers = headers)
    response.raise_for_status()
    return response

def post (url, params=None, extend=True):
    global base_url, token
    set_token()
    if extend == True:
        url = base_url + url
    headers = {'Content-Type': 'application/json', 'Authorization': f'Token {token}'}

    response = requests.post(url, headers = headers, data=params)
    response.raise_for_status()
    return response


def set_token ():
    global token
    global token_timestamp

    try:
        token_timestamp
    except NameError:
        token_timestamp = 0

    if token_timestamp == 0:
        param = {
            'username' : cfg['username'],
            'password' : cfg['password']
        }
        headers = {'Content-Type': 'application/x-www-form-urlencoded'}
        url = cfg['authurl']
        res = requests.post(url, data=param)
        res.raise_for_status()
        token_timestamp = time.time()
        token = res.json()['token']



res = get('ambulance')  

# if __name__ == "__main__":
#   main()

In [ ]:
ambulances = res.json()

In [ ]:
vehicles = {}
df = pd.DataFrame()
for ambulance in ambulances:
    id = ambulance['id']
    data = get(f'ambulance/{id}/updates/').json()#?filter=2020-10-28T15:00:00.000Z,2020-10-30T15:00:00.000Z').json() 
    data = [{'id':id, 'identifier':ambulance['identifier'], **item} for item in data]
    df = df.append(pd.DataFrame(data))
    #print(data) 
    vehicles[id] = {'ambulance':ambulance, 'data':data}
 
    
     #Just doing for one for now


In [ ]:
def splitlotlan(row):
    row['lon'] = row['location']['longitude']
    row['lat'] = row['location']['latitude']
    if row['lat'] == 37.4219983 and row['lon'] == -122.084:
        return None
    return row
  
df = df.apply(splitlotlan, axis=1).dropna()

In [ ]:
m = folium.Map(
    location=list(df[['lat', 'lon']].mean().values.tolist()),
    zoom_start=16
)
m.fit_bounds(
    [df[['lat', 'lon']].min().values.tolist(), df[['lat', 'lon']].max().values.tolist()]
)
# Lon, Lat order.
lines = []
dat = list(zip(df['lon'],df['lat'],df['timestamp']))
for i in range (1, len(dat)):
    lines.append({
      'coordinates': [
            list(dat[i][:2]),
            list(dat[i-1][:2]),
        ],
        'dates': [
            dat[i][2],
            dat[i-1][2]
        ],
        'color': 'red'
  })
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'LineString',
            'coordinates': line['coordinates'],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': line['weight'] if 'weight' in line else 5
            }
        }
    }
    for line in lines
]

plugins.TimestampedGeoJson({
    'type': 'FeatureCollection',
    'features': features,
}, period='PT1M', add_last_point=True).add_to(m)
m.save(os.path.join( 'sample.html'))

m

In [ ]:
import plotly.graph_objects as go # or plotly.express as px
 # or any Plotly Express function e.g. px.bar(...)
# fig.add_trace( ... )
# fig.update_layout( ... )

import dash
import dash_core_components as dcc
import dash_html_components as html

import plotly.express as px


In [ ]:

fig = px.line_mapbox(df[df['identifier']=='7BST414'], lat="lat", lon="lon", color="identifier", zoom=3, height=500)

fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=4, mapbox_center_lat = 41,
    margin={"r":0,"t":0,"l":0,"b":0})

fig.show()



In [ ]:
data = df[df['identifier']=='7BST414']
data['timestamp'] = pd.to_datetime(data.timestamp)
data = data.sort_values(by='timestamp', ascending=True).reset_index()
center = {'lon': data['lon'].mean(), 'lat':data['lat'].mean()}
fig = go.Figure()

for step in range(2,data.shape[0],5):
    fig.add_trace(
        go.Scattermapbox(
        visible=False,
        mode = "markers+lines",
        lon = data.loc[:step:5,'lon'],
        lat = data.loc[:step:5,'lat'],
        marker = {'size': 10}))
          
        

        
fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': center,
        'style': "stamen-terrain",
        'zoom': 8}
)

fig.data[10].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Slider switched to step: " + str(i)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Frequency: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

fig.show()
fig.write_html("plotly.html")
